In [63]:
import pandas as pd
import numpy as np
eps = np.finfo(float).eps
from numpy import log2 as log
import sys

In [64]:
sys.setrecursionlimit(10**6)

dataset = pd.read_csv('bank.csv')

df = dataset.loc[:300]

In [65]:
def find_entropy(df):
    Class = df.keys()[-1]  
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value] / len(df[Class])
        entropy += -fraction * np.log2(fraction)
    return entropy

In [66]:
def find_entropy_attribute(df, attribute):
    Class = df.keys()[-1] 
    target_variables = df[Class].unique() 
    variables = df[attribute].unique()
    
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute] == variable][df[Class] == target_variable])
            den = len(df[attribute][df[attribute] == variable])
            fraction = num / (den + eps)
            entropy += -fraction * log(fraction + eps)
        fraction2 = den / len(df)
        entropy2 += -fraction2 * entropy
    return abs(entropy2)

In [67]:
def find_winner(df):
    IG = []
    for key in df.keys()[:-1]:
        IG.append(find_entropy(df) - find_entropy_attribute(df, key))
    return df.keys()[:-1][np.argmax(IG)]

In [69]:
def buildTree(df, tree=None):
    node = find_winner(df)

    attValue = np.unique(df[node])

    if tree is None:
        tree = {}
        tree[node] = {}

    for value in attValue:

        subtable = get_subtable(df, node, value)
        clValue, counts = np.unique(subtable['loan'], return_counts=True)

        if len(counts) == 1: 
            tree[node][value] = clValue[0]
        else:
            tree[node][value] = buildTree(subtable) 

    return tree

In [70]:
tree=buildTree(df)
import pprint
pprint.pprint(tree)

{'day': {2: {'marital': {'divorced': 'no',
                         'married': {'job': {'blue-collar': {'education': {'primary': {'loan': {'no': 'no',
                                                                                                'yes': 'yes'}},
                                                                           'secondary': 'no'}},
                                             'management': 'no',
                                             'services': 'no',
                                             'technician': 'no'}},
                         'single': 'no'}},
         3: {'job': {'admin.': 'no',
                     'blue-collar': 'no',
                     'entrepreneur': 'no',
                     'management': 'no',
                     'self-employed': 'yes',
                     'services': 'no',
                     'technician': 'no',
                     'unemployed': 'yes'}},
         4: {'job': {'admin.': 'no',
                     'blue-collar'

In [71]:
def predict(inst,tree):

    for nodes in tree.keys():        
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
            
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction

In [73]:
test_data =  dataset.loc[50:60]
test_data = test_data.to_numpy()
for i in range(len(test_data)):
    new_dict = {'job':test_data[i][0], 'martial':test_data[i][1], 'education':test_data[i][2],
                'default':test_data[i][3], 'housing':test_data[i][4],'day':test_data[i][6],
                'month':test_data[i][7]
               }
    inst = pd.Series(new_dict)
    prediction = predict(inst,tree)
    print('Prediction : ', prediction)

Prediction :  no
Prediction :  yes
Prediction :  yes
Prediction :  no
Prediction :  no
Prediction :  no
Prediction :  no
Prediction :  no
Prediction :  no
Prediction :  no
Prediction :  no
